In [ ]:
import csv
from matplotlib import pyplot as plt
from matplotlib import cm
import numpy as np
from scipy.interpolate import make_interp_spline, BSpline, make_lsq_spline
from scipy.optimize import curve_fit

# Colormaps: paired
# Extract colors from colormap, to use at will
colormap = cm.get_cmap('Paired')
colors = [colormap(i) for i in np.linspace(0, 1, 12)]

# Define fitting function
#def fitting_exponential(x, y0, v0, k):
#    return y0 - (v0/k) * (1 - np.exp(-k*x))

def fitting_exponential(x, a, c, d):
    return a*np.exp(-c*x)+d

def fitting_sigmoid(x, a, b, c, d):
    return d + (a - d)/(1 + np.power(x / c, b))

# Wrappers for easier use of interpolate functions from scipy
def generate_interp_spline(time, product, time_smooth):
    time = time[:len(product)]
    spl = make_interp_spline(time, product, k=3)  # type: BSpline
    smooth_data = spl(time_smooth)
    return smooth_data

def generate_lsq_spline(time, product, time_smooth):
    time = time[:len(product)]
    popt, pcov = curve_fit(fitting_exponential, time, product)
    print(popt)
    smooth_data = fitting_exponential(time_smooth, *popt)
    return smooth_data

In [ ]:
PNO_time = dict([("1%", []), ("2%", []), ("5%", []), ("5%_noH2O", [])])
PNO_yield = dict([("1%", []), ("2%", []), ("5%", []), ("5%_noH2O", [])])

with open('./data/PNO_Optimisation.csv', 'r') as data_file:
    plots = csv.reader(data_file, delimiter=';')
    next(plots)
    next(plots)
    next(plots)
    for row in plots:
        PNO_time["5%_noH2O"].append(row[0])
        PNO_yield["5%_noH2O"].append(row[1])
        PNO_time["1%"].append(row[2])
        PNO_yield["1%"].append(row[3])
        PNO_time["2%"].append(row[4])
        PNO_yield["2%"].append(row[5])
        PNO_time["5%"].append(row[6])
        PNO_yield["5%"].append(row[7])

# Cast str to float
PNO_yield = {j:[float(i) for i in PNO_yield[j] if i] for j in PNO_yield}
PNO_time = {j:[float(i) for i in PNO_time[j] if i] for j in PNO_time}

# Smoothen data for fitting
time_smooth = np.linspace(0, 48, 300)
smooth_data = dict([("1%", []), ("2%", []), ("5%", []), ("5%_noH2O", [])])

# Fittings form origin, Model ExpGro1
smooth_data["1%"] = [81.35845*np.exp(-4.78108/(x+1.22669)) for x in time_smooth]
smooth_data["2%"] = [np.exp(4.36918 + -3.27342/(x+0.9801)) for x in time_smooth]
smooth_data["5%"] = [-84.96408*np.exp(-x/4.97392) + 84.96408 for x in time_smooth]
smooth_data["5%_noH2O"] = [-47.14067*np.exp(-x/9.29703) + 47.14067 for x in time_smooth]


In [ ]:
# Setup axes
axis = plt.gca()
axis.set_axisbelow(True)
axis.set_xlabel("Time (min)")
axis.set_xlim(0, 35)
axis.set_ylim(-5, 100)
axis.set_ylabel("Yield (%)")

# Plot lines

plt.plot(PNO_time['1%'], PNO_yield['1%'],'o' , zorder=2.6, color=colors[5], label="1%")
plt.plot(time_smooth, smooth_data['1%'],'-' , zorder=1.6, color=colors[4])
plt.plot(PNO_time['2%'], PNO_yield['2%'],'o' , zorder=2.6, color=colors[1], label="2%")
plt.plot(time_smooth, smooth_data['2%'],'-' , zorder=1.6, color=colors[0])
plt.plot(PNO_time['5%'], PNO_yield['5%'],'o' , zorder=2.6, color=colors[3], label="5%")
plt.plot(time_smooth, smooth_data['5%'],'-' , zorder=1.6, color=colors[2])
plt.plot(PNO_time['5%_noH2O'], PNO_yield['5%_noH2O'],'o' , zorder=2.6, color=colors[7], label="5%, no H$_2$O")
plt.plot(time_smooth, smooth_data['5%_noH2O'],'-' , zorder=1.6, color=colors[6])

plt.legend(fontsize='small')
plt.tight_layout()
# Save file
plt.savefig("./data/PNO_Optimization.jpg", dpi=300)